# SBP Operators
#### References:

Carpenter, M. H., Parsani, M., Fisher, T. C., & Nielsen, E. J. (2015). Entropy Stable Staggered Grid Spectral Collocation for the Burgers' and Compressible Navier-Stokes Equations (No. NF1676L-22150).

Canuto, C., Hussaini, M. Y., Quarteroni, A., & Zang, T. A. (2007). Spectral methods: fundamentals in single domains. Springer Science & Business Media.


## Computing Lagrange basis

We import some useful modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from basis_nodes import generate_lagrange_poly, generate_LGL_points

The function generate_lagrange_poly(j, x_nodes) will return a Lagrange polynomial and its derivative:

$$
L_j(x)=\prod_{k \neq j} \frac{x-x_k}{x_j-x_k},
$$
and
$$
L_j^{\prime}(x)=\frac{\sum_{k=0, k \neq j}^n \prod_{l=0, l \neq k, l \neq j}^n\left(x-x_l\right)}{\prod_{k=0, k \neq j}^n\left(x_j-x_k\right)}.
$$

A set of nodes should be provided, it is desired that these be the Legendre-Gauss-Lobatto (LGL) nodes.

In [ ]:
nodes = np.array([0,1,2,3,4,5,6])
idx = 4
poly, polyp = generate_lagrange_poly(idx,nodes)

In [ ]:
vpoly = np.vectorize(poly)
vpolyp = np.vectorize(polyp)

In [ ]:
x = np.linspace(0,6,50)

fig = plt.figure(dpi=100, figsize=(8,4))
ax = fig.add_subplot(1, 1, 1)
ax.set_ylim([-5,5])
ax.plot(x,np.zeros(len(x)),color='k', lw=1)
ax.plot(x, vpoly(x), label=f'$L_{idx}(x)$')
ax.plot(x, vpolyp(x),color='g', label=f'$L_{idx}\'(x)$')
ax.scatter(nodes,vpoly(nodes),marker='x', color='r')
plt.legend()
plt.show()

## Computing LG and LGL points

In [ ]:
n_points = 10
(p_Legn, p_Legn_prime,
p_Legn1, p_Legn1_prime,
x_LG, w_LG,
x_LGL, w_LGL) = generate_LGL_points(n_points)
x = np.linspace(-1,1,1000)

The Legendre polynomials are computed as eigenfunctions of the Sturm-Liouville problem (Canuto et al., 2006):

$$
((1-x^2) L_k'(x))'+k(k+1) L_k(x) = 0.
$$

The Legendre-Gauss nodes and weights are computed as
$$
\begin{gathered}
x_j(j=0, \ldots, N) \text { zeros of } L_{N+1} ; \\
w_j=\frac{2}{\left(1-x_j^2\right)\left[L_{N+1}^{\prime}\left(x_j\right)\right]^2}, \quad j=0, \ldots, N
\end{gathered}
$$

In [ ]:
fig = plt.figure(dpi=100, figsize=(8,3))
ax = fig.add_subplot(1, 1, 1)
ax.plot(x,np.zeros(len(x)),color='k', lw=1)
ax.plot(x, p_Legn1(x), label=f'$L(x)$')
ax.scatter(x_LG,np.zeros(len(x_LG)),marker='x', color='r', label='LG points')
#ax.scatter(x_LG,w_LG,marker='x', color='g', label='LGL weights')
plt.legend()
plt.show()

The Legendre-Gauss-Lobatto nodes and weights are computed as

$$
\begin{gathered}
x_0=-1, x_N=1, x_j(j=1, \ldots, N-1) \text { zeros of } L_N^{\prime} ; \\
w_j=\frac{2}{N(N+1)} \frac{1}{\left[L_N\left(x_j\right)\right]^2}, \quad j=0, \ldots, N .
\end{gathered}
$$

In [ ]:
#tol=1.e-14
#findiff=(p_Legn(x+tol)-p_Legn(x-tol))/tol
fig = plt.figure(dpi=100, figsize=(8,3))
ax = fig.add_subplot(1, 1, 1)
ax.plot(x,np.zeros(len(x)),color='k', lw=1)
ax.plot(x, p_Legn_prime(x), label=f'$L\'(x)$')
#ax.plot(x,findiff, color='g')
ax.scatter(x_LGL,np.zeros(len(x_LGL)),marker='x', color='r', label='LGL points')
#ax.scatter(x_LGL,w_LGL,marker='x', color='g', label='LGL weights')
plt.legend()
plt.show()

Validating nodes and weights with some simple integrals

In [ ]:
f = lambda x: x**2
g = lambda x: (1./3.)*x**3
num_int_LG=np.sum(f(x_LG)*w_LG)
num_int_LGL=np.sum(f(x_LGL)*w_LGL)
exact_int = g(1)-g(-1)
print(f"LG integration: {num_int_LG}")
print(f"LGL integration:{num_int_LGL}")
print(f"Exact integral: {exact_int}")


## Computing first order SBP matrices

In [ ]:
from SBP_matrices import first_order_D, first_order_P_Q

n_points = 2
(p_Legn, p_Legn_prime,
p_Legn1, p_Legn1_prime,
x_LG, w_LG,
x_LGL, w_LGL) = generate_LGL_points(n_points)

n_nodes = len(x_LGL)
x = np.linspace(-0.5,0.5,n_nodes)
D_LG = first_order_D(x_nodes=x_LG)
D_LGL = first_order_D(x_nodes=x_LGL)
D = first_order_D(x_nodes=x)
print("D_LG:")
print(np.round(D_LG,2))
print("D_LGL:")
print(np.round(D_LGL,2))
print("D:")
print(np.round(D,2))

In [ ]:
P_LG, Q_LG = first_order_P_Q(x_Lagrange_nodes=x_LG)
P_LGL, Q_LGL = first_order_P_Q(x_Lagrange_nodes=x_LGL)
P, Q = first_order_P_Q(x_Lagrange_nodes=x)
print("P_LG:")
print(np.round(P_LG,2))
print("P_LGL:")
print(np.round(P_LGL,2))
print("P:")
print(np.round(P,2))

### Validating our computations using Q
We compute Q through 
$$
Q = \sum_{\ell} {\bf L}(\eta_{\ell};{\bf x}) {\bf L}(\eta_{\ell};{\bf x})^T \omega_{\ell},
$$
and 
$Q=PQ$.

In [ ]:
print("P_LG @ D_LG:")
print(np.round(P_LG@D_LG,2))
print("P_LGL @ D_LG:")
print(np.round(P_LGL@D_LGL,2))
print("P @ D:")
print(np.round(P@D,2))

In [ ]:
print("Q_LG:")
print(np.round(Q_LG,2))
print("Q_LGL:")
print(np.round(Q_LGL,2))
print("Q:")
print(np.round(Q,2))

### Verifying SBP properties
From inspection we can see that $P$ is symmetric for the different Lagrange bases, lets check if it's PD:

In [ ]:
def is_pos_def(A):
    if np.array_equal(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False
print(f"P_LG SPD?: {is_pos_def(P_LG)}")
print(f"P_LGL SPD?: {is_pos_def(P_LGL)}")
print(f"P SPD?: {is_pos_def(P)}")

Now we check if
$$
    Q^T = B- Q. \qquad B=\text{Diag}(-1,0,...,0,1)
$$

In [ ]:
B=np.zeros((len(P),len(P))); B[0,0]=-1; B[-1,-1]=1
print(f"Q_LG^T - (B-Q_LG):\n", np.round(np.transpose(Q_LG)-(B-Q_LG),2))
print(f"Q_LGL^T - (B-Q_LGL):\n", np.round(np.transpose(Q_LGL)-(B-Q_LGL),2))
print(f"Q^T - (B-Q):\n", np.round(np.transpose(Q)-(B-Q),2))

Apart from having a diagonal matrix $P_{LGL}$, the Lagrange basis produced by the LGL nodes gives us matrices $P_{LGL}$ and $Q_{LGL}$ that satisfy the SBP convention.